Richard,

Thanks for the article. Here are some problems I have with that interpretation of Shcrodinger's cat 

In [41]:
from __future__ import division

In [1]:
import random

In [17]:
#Give me a random percentage between 0 and 100
a = random.randint(1,100)

Let's assume we have the typical Schrödinger's cat experiment. To save on cats, we'll instead use the function experiment() to give a result of either 1, for alive, or 0, for dead. experiment() will generate these result randomly with a probability of 50% for each result.

In [40]:
trials = 50
count = 0

for x in range(trials):
    prob_of_truth = 50 + 100*(x/(2*trials))
    print(prob_of_truth)
    a = random.randint(1,100)
    if a<prob_of_truth:
        truth(result)
    else:
        lie(result)
print (count / trials)

50.0
The cat is dead
51.0
The cat is dead
52.0
The cat is dead
53.0
The cat is alive
54.0
The cat is dead
55.0
The cat is dead
56.0
The cat is dead
57.0
The cat is alive
58.0
The cat is alive
59.0
The cat is dead
60.0
The cat is alive
61.0
The cat is dead
62.0
The cat is dead
63.0
The cat is alive
64.0
The cat is alive
65.0
The cat is dead
66.0
The cat is dead
67.0
The cat is dead
68.0
The cat is alive
69.0
The cat is dead
70.0
The cat is alive
71.0
The cat is dead
72.0
The cat is alive
73.0
The cat is alive
74.0
The cat is alive
75.0
The cat is alive
76.0
The cat is dead
77.0
The cat is alive
78.0
The cat is alive
79.0
The cat is alive
80.0
The cat is alive
81.0
The cat is dead
82.0
The cat is alive
83.0
The cat is alive
84.0
The cat is alive
85.0
The cat is alive
86.0
The cat is alive
87.0
The cat is alive
88.0
The cat is alive
89.0
The cat is alive
90.0
The cat is alive
91.0
The cat is dead
92.0
The cat is alive
93.0
The cat is alive
94.0
The cat is alive
95.0
The cat is alive
96.0


In [23]:
def truth(result):
    if result==1:
        print("The cat is alive")
    else:
        print("The cat is dead")
def lie(result):
    if result==1:
        print("The cat is dead")
    else:
        print("The cat is alive")

In [25]:
def experiment():
    alive = random.randint(0,1)
    return alive

In [30]:
for x in range(5):
    print(experiment())

0
0
1
0
1


In [32]:
result = experiment()

In [ ]:
#The other person learns whether the truth or a lie was told

In [22]:
#Two people could also lie to each other with a probability